# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import os

from glob import glob
from tqdm.auto import tqdm

# Data Load
- 데이터 비식별화 작업을 수행 후 데이터 전처리 수행 가능

In [ ]:
root_path = "./Ubiosis/data" # 데이터 저장 폴더 경로

# 1. 학습 대상자 폴더 정합성 확인
## 운영체제 특성 또는 외장하드 특성상 숨김 파일 또는 임시 파일까지 읽어들이는 경우가 있어서 필터링 필요
tmp_subject_list = list(os.listdir(root_path)) # 데이터 저장 폴더 안에 있는 파일 및 폴더명을 불러와서 리스트에 저장
subject_list = [] # 정상 대상자 id 저장할 리스트 객체
for i in range(len(tmp_subject_list)):
    subject = tmp_subject_list[i] # 리스트 객체의 원소 1개씩 탐색
    first_char = subject[0] # 폴더 및 파일 명의 첫번째 문자를 가져옴
    # 비식별화를 수행하였으면, 폴더의 명이 u 로 시작하기 때문에 첫번째 문자가 u 로 시작하는 폴더명을 정상 대상자 id 리스트 객체에 저장
    if first_char == 'u':
        subject_list.append(subject)

# 2. 정상 대상자 id 리스트에서 데이터 추출
data_list = [] # 데이터를 담을 리스트 객체 정의
err_list = [] # 데이터에 오류가 발생 시 오류 대상자를 담을 리스트 정의
for idx in tqdm(range(len(subject_list))):
    subject = subject_list[idx] # 대상자 1명씩 탐색
    subject_path = os.path.join(root_path, subject) # 대상자 폴더의 경로 문자열
    try:
        datas = [] # 대상자 1명의 데이터를 담을 리스트 객체 정의
        # dat 파일에서 추출한 Radius 정보와 CIS 시작 위치를 저장하였던 meta csv 파일 호출
        meta_file = pd.read_csv(os.path.join(subject_path, "_".join((subject, "meta.csv"))))
        start_idx = meta_file['FIRST_TEST_COUNT'].values[0]
        radius_value = meta_file['RISER RADIUS'].values[0]
        
        # Radius 값 변환
        ## X / 2 / 1000 * 0.9473677 = Radius value
        ## X = Radius value * 2 * 1000 / 0,9473677
        r_radius = round(float(radius_value) / 0.9473677 * 1000 * 2, 2)
        # Radius의 변환 값이 3.1이상 3.28이하인 경우에만 CIS 데이터 추출 수행
        ## 이후 Radius의 기준 범위가 변경될 시 조건문의 3.1과 3.28의 범위 수정 혹은
        ## 새로운 조건을 elif 문을 통해 추가 필요
        if r_radius >= 3.1 and r_radius <= 3.28:
            # OCR을 통해 저장하였던 shear rate 값 호출
            y_file = pd.read_csv(os.path.join(subject_path, "_".join((subject, "y.csv"))))
            y_file = y_file.astype('float32')
            
            v_1000 = float(y_file['1000'].iloc[0])
            v_300 = float(y_file['300'].iloc[0])
            v_150 = float(y_file['150'].iloc[0])
            v_100 = float(y_file['100'].iloc[0])
            v_50 = float(y_file['50'].iloc[0])
            v_10 = float(y_file['10'].iloc[0])
            v_5 = float(y_file['5'].iloc[0])
            v_2 = float(y_file['2'].iloc[0])
            v_1 = float(y_file['1'].iloc[0])
            v_list = [v_1000, v_300, v_150, v_100, v_50, v_10, v_5, v_2, v_1]

            # Shear Rate의 값이 정상 범주 안에 있는 경우의 데이터만 추출하기 위한 조건문
            # Shear Rate의 정상 값의 범위 변경 시 해당 if 문의 2와 6의 값 변경 필요
            ## Shear Rate의 정상 값의 범위에 해당하는 CIS 데이터 추출
            if v_1000 >= 2 and v_1000 < 6:
                x_file = pd.read_csv(os.path.join(subject_path, "_".join((subject, "x.csv"))))
                x_file = x_file.astype('float32')
                cis_1_data = list(x_file["CIS_1"].values[start_idx:start_idx+6000])
                cis_2_data = list(x_file["CIS_2"].values[start_idx:start_idx+6000])
                
                if len(cis_1_data) >= 6000 and len(cis_2_data):
                    datas.extend([r_radius])
                    datas.extend(cis_1_data)
                    datas.extend(cis_2_data)
                    datas.extend(v_list)
                    
                    data_list.append(datas)
                else:
                    pass
            else:
                pass
        
    except Exception as ex:
        # print(subject_path)
        err_list.append(subject)
        pass

In [ ]:
# 데이터프레임(csv) 파일의 헤더 생성
col_list = ["RADIUS"]
for i in range(1,6001):
    cc = "_".join(("CIS_1", str(i)))
    col_list.append(cc)
for i in range(1,6001):
    cc = "_".join(("CIS_2", str(i)))
    col_list.append(cc)
col_list.extend(["1000","300","150","100","50","10","5","2","1"])

# 추출한 데이터의 정보를 통해 데이터프레임 구성
data_df = pd.DataFrame(columns=col_list,data=data_list)

In [ ]:
# 학습을 위한 데이터 저장
file_nm = "./Ubiosis/total_data.csv"
data_df.to_csv(file_nm, index=False)